# 复习一下之前的

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
#
# 从checkpoint自动加载 分词器和模型
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [7]:
sequence = "I've been waiting for a HuggingFace course my whole life."

In [8]:
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor(ids)
# This line will fail.
model(input_ids)

IndexError: too many indices for tensor of dimension 1

In [9]:
# 失败了！！！ 
# HF
# 问题在于我们向模型发送了一个序列，而 🤗 Transformers 模型默认需要多个句子。
# 在这里，我们尝试执行 tokenizer 在应用于一个序列时在幕后所做的所有操作。
# 但是，如果你仔细观察，你会发现 tokenizer 不仅将输入 ID 列表转换为张量，还在其顶部添加了一个维度：
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])


In [10]:
# 
# 进行正确的操作
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequence = "I've been waiting for a HuggingFace course my whole life."

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)

# 增加一个维度
input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


In [12]:
#
# 试一下两个
batched_ids = torch.tensor([ids, ids])
output_2 = model(batched_ids)
output_2

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7276,  2.8789],
        [-2.7276,  2.8789]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

# Padding the inputs  填充输入

In [ ]:
#
# HF
# 以下列表列表无法转换为 Tensor：
batched_ids = [
    [200, 200, 200],
    [200, 200]
]
# 为了解决这个问题，我们将使用填充使我们的张量具有矩形形状。
# Padding 通过向值较少的句子添加一个名为 padding token 的特殊单词来确保我们所有的句子具有相同的长度。
# 例如，如果您有 10 个句子包含 10 个单词，1 个句子包含 20 个单词，则填充将确保所有句子都包含 20 个单词。
# 在我们的示例中，生成的张量如下所示：
padding_id = 100

batched_ids = [
    [200, 200, 200],
    [200, 200, padding_id],
]

In [ ]:
# 填充令牌 ID 可以在 tokenizer.pad_token_id 中找到。
# 让我们使用它，将我们的两个句子单独发送到模型中，并一起批处理：

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)
# HF
# 我们的批量预测中的 logit 有问题：第二行应该与第二句的 logit 相同，但我们有完全不同的值！
# 这是因为 Transformer 模型的关键特征是将 每个 Token 置于  上下文   中的注意力层。
# 这些将       考虑 padding token，      因为它们会处理 sequence 的所有 token。
# 为了在通过模型传递不同长度的单个句子时，或者在传递应用了相同句子和填充的批处理时获得相同的结果，
# 我们需要告诉这些注意力层忽略填充标记。这是通过使用    ！！！！注意力掩码来！！！    完成的。

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


# Attention masks  注意蒙版

In [14]:
#
# HF
# 注意力掩码是与输入 ID 张量形状完全相同的张量，填充有 0 和 1：1 表示应该关注相应的令牌，0 表示不应关注相应的令牌

In [16]:
# 让我们使用注意力掩码完成前面的示例：
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


# 更长的序列

In [17]:
截断 或 换模型

SyntaxError: invalid syntax (1431207749.py, line 1)